# HW 2

For this homework you will be using pytorch and torchvision library for neural networks and datasets. You can install them with pip install torch torchvision.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
import pandas as pd

# 1 Question 1 Principal Component Analysis

This problem will guide you through the principal component analysis. You will be using a classical dataset, the MNIST hand written digit dataset.

In [ ]:
 # Load the MNIST dataset
mnist = MNIST('.', download=True)
data = mnist.train_data.numpy()
labels = mnist.train_labels.numpy()
print('shapes:', data.shape, labels.shape)
plt.imshow(data[0])
print('label:', labels[0])

### 1.1 Step 1.1 Familiarize yourself with the data

For this task, you will be using the torchvision package that provides the MNIST dataset. For each digit class(0-9), plot 1 image from the class and store those 10 images for each digit class in the array digit_images.

In [ ]:
digit_images = np.zeros([10, 28, 28])
for i in range(10):
  for x,y in zip(data, labels):
    if y == i:
      digit_images[i] = x
      break
  plt.imshow(digit_images[i])
  plt.show()


## 2 Question 2. PCA

This section deals with performing PCA on the MNIST dataset. We will explore 2 different methods of performing PCA. First using SVD, second using Eigen Decomposition. Please refer to the lecture slides, in particular slides 30-32 from Lec 3 which can be found [here](https://ucsd-cse-152.github.io/FA20/slides/Lec3.pdf).

### 2.1 Question 2.1 Centering the data [5 points]

For each image, flatten it to a 1-D vector. To perform PCA on the dataset, we first move the data points so they have 0 mean on each dimension. Store the centered data in variable **data_centered** and the mean of each dimension in variable **data_mean**.


In [ ]:
flattened_d= np.array([data_f.flatten() for data_f in data]) #flatten matrix
data_mean = np.mean(flattened_d, axis = 0) 
data_centered = flattened_d - data_mean

print(flattened_d.shape)

# hint: You need np.mean() and x.reshape() functions (x is a matrix). 
### END OF CODE

### 2.2 Compute the Singular Value Decomposition (SVD) for the given data [5 points]


In [ ]:
U,S,V = None, None, None
### YOUR CODE HERE

U,S,V = np.linalg.svd(data_centered, full_matrices = False) #S is shape (60000,28)
print(U,S,V)
### END OF CODE

### Question 2.3 Project data onto the first 2 principal components [5 points]

Now you need to project the centered data on the 2D space formed by the eigenvectors corresponding to the 2 largest eigenvalues. Create a 2D scatter plot where you need to assign a unique color to each digit class.

In [ ]:
import seaborn as sns 
# this is the library for visualization
# you can use sns.scatterplot() function to visualize a scattered plot of the results

### YOUR CODE HERE

S = np.square(S)
#take either 2 first columns of U or 2 first rows of V
eigv = [V[0], V[1]]
projected = np.asmatrix(eigv) @ data_centered.T



### END OF CODE

df = pd.DataFrame(data = projected, columns=['comp1', 'comp2'])
df['labels'] = pd.Series(labels)
sns.scatterplot(
    x="comp1", y="comp2",
    hue="labels",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.3
)


### Question 2.4 Unproject data back to high dimensions [5 points]
For this question, you need to project the 10 images you plotted in **1.2.3** on the first 2 principal components, and then unproject the "compressed" 2-D representations back to the original space. Plot the "compressed" digit (the reconstructed digit). Do they look similar to the original images?

In [ ]:
### YOUR CODE HERE
eigvectors = [V[i] for i in range(20)]
#tranform the test images
reduced_test_image = (current_test_image - data_mean) @ eigvectors 
#transform them back 
reconstructed = (reduced_test_image @ eigenvectors.T) + data_mean
### END OF CODE

digit_images = np.zeros([10, 28, 28])
for i in range(10):
  for x,y in zip(reconstructed, labels):
    x = x.reshape(28,28)
    if y == i:
      digit_images[i] = x
      break
  plt.imshow(digit_images[i])
  plt.show()



### Question 2.5 Choose a better low dimension space. [5pt]
Do the previous problem with more dimensions (e.g. 3, 5, 10, 20, 50, 100). You only need to show results for one of them. Answer the following questions. How many dimensions are required to represent the digits reasonably well? Briefly explain what criteria you used to select the number of dimensions and how it relates to the SVD.

### Question 2.6 PCA Using Eigen Decomposition

Now we will perform PCA using method 2: Eigen Decomposition.

For this method, we need to compute the covariance matrix of the data first. Prof. Su has added three slides at the end of Lec 4 (https://ucsd-cse-152.github.io/FA20/slides/Lec3.pdf) from P30-P32. Please read the slides before you proceed. 

### Question 2.6.1 Compute the covariance matrix of the data [5 points]
You need to store the covariance matrix of the data in variable data_covmat. You may not use numpy.cov

In [ ]:
data_covmat = None
### YOUR CODE HERE
data_covmat =  1/len(data_centered)*data_centered.T @ data_centered
### END OF CODE

### Question 2.6.2 Compute the eigenvalues and eigen vectors of the covariance matrix [5pt]
You need to store the eigenvalues of the covariance matrix in variable `covmat_eig`, sorted in descending order and the corresponding eigenvectors in `covmat_eigvec`. Then you need to plot the eigenvalues with `plt.plot`. You can use any numpy function.

In [ ]:
covmat_eigvec, covmat_eig = None, None
### YOUR CODE HERE
# hint: you may use np.linalg.eig
covmat_eig, covmat_eigvec = np.linalg.eig(data_covmat)
idx = np.argsort(covmat_eig)[::-1]
evecs = covmat_eigvec[:,idx]
eig = covmat_eig[idx]
plt.plot(eig)
plt.xlabel("dimension")
plt.ylabel("eigenvalues")
### END OF CODE

### Question 2.6.3 Project data onto the first 2 principal components [5 points]

Now you need to project the centered data on the 2D space formed by the eigenvectors corresponding to the 2 largest eigenvalues. Create a 2D scatter plot where you need to assign a unique color to each digit class. Feel free to use any plotting libraries

In [ ]:
import seaborn as sns
### YOUR CODE HERE
projected = None
import matplotlib.cm as cm
data_PCA = covmat_eigvec[:,:2].T @ data_centered.T
colors = cm.rainbow(np.linspace(0, 1, 10))
for i in range(10):
    l = np.where(labels == i)
    plt.scatter(data_PCA[0][l],data_PCA[1][l], s = 0.1, c = colors[i]) 
### END OF CODE
df = pd.DataFrame(data = projected, columns=['comp1', 'comp2'])
df['labels'] = pd.Series(labels)
sns.scatterplot(
    x="comp1", y="comp2",
    hue="labels",
    palette=sns.color_palette("hls", 10),
    data=df,
    legend="full",
    alpha=0.3
)

### Question 2.6.4 Compare the 2 approaches

What if any is the difference/similariy in computing PCA between the 2 approaches ? Briefly justify your answer.

## Question 3: Neural Networks for Regression

As an introduction to the idea of performing regression using Neural Networks, we will first try to build a basic MultiLayer Perceptron (MLP) for fitting a line to random data. 

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data


torch.manual_seed(1)    # reproducible

x = torch.unsqueeze(torch.linspace(-1, 1, 100), dim=1)  # x data (tensor), shape=(100, 1)
y = x.pow(2) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)

# view data
plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### 3.1 Fitting a basic model

Let's try to fit a basic 2 layer network for this given data. We will use one hidden layer with 10 neurons/units and the output layer has one neuron to predict a single value. For this given data, given the x value our goal is to predict a y value which most accurately describes the data.

The basic pytorch work flow involves 4 steps:
- Implement the model and initialize it
- Decide on the right optimizer
- Choose an appropriate loss function for your learning task
- train the model over the given data

In [ ]:
# this is one way to define a network
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(n_feature=1, n_hidden=10, n_output=1)     # define the network
print(net)  # net architecture

For this simple model we will be using the MSE Loss which is the most common and simple loss function used for regression tasks. The MSE loss is the l2 norm of the difference. 
The MSE loss is given by:
$$
L(x,y) = \frac{1}{N}\Sigma_{i=1}^{i=n} (x - y)^2
$$
where $N$ is the number of data points

For our optimizer we will use Stochastic Gradient Descent (SGD) which is sufficient for this problem

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
loss_func = torch.nn.MSELoss()

We now train the network on the data for 200 epochs. The training process involves first forward propogating the input through the network after which we calculate the loss value from the current prediction. We then backpropogate the gradient of the loss all the way back through the network. This is done by the `loss.backward()` statement. Once the gradients have been computed we update the weights which in pytorch can be easily done by calling the `optimizer.step()` function.

In [ ]:
# train the network
for t in range(200):
    prediction = net(x)     # input x and predict based on x
    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)
    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

Let's compare how our model performs by plotting the line fit by our basic MLP on top of the data that we generated

In [ ]:
fig, ax = plt.subplots(figsize=(12,7))    
ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange", label='ground truth')
ax.plot(x.data.numpy(), prediction.data.numpy(), 'g-', lw=3, label='prediction')
plt.legend()
plt.show()

### Q3.2 Fit a model for the sin function

We tried to fit a model to a basic randomly generated dataset. Let's now try to implement a MLP that can fit the sin function. We start once again by generating a dataset for this.

In [ ]:

x = torch.unsqueeze(torch.linspace(-10, 10, 1000), dim=1)  # x data (tensor), shape=(100, 1)
y = torch.sin(x) + 0.2*torch.rand(x.size())                 # noisy y data (tensor), shape=(100, 1)

x, y = Variable(x), Variable(y)

plt.figure(figsize=(10,4))
plt.scatter(x.data.numpy(), y.data.numpy(), color = "orange")
plt.title('Regression Analysis')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### Q 3.2 Implement the MLP [5 points]




Fill in the template code below to configure your model. Implement the model description as well as the forward pass for the network. 

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
      super(Net, self).__init__()
      
      self.fc1 = torch.nn.Linear(1, 200)
      # ... continue to construct the network
      ### YOUR CODE HERE
      self.layer1 = nn.Linear(input_size, hidden_size)
        # activation
      self.relu = nn.ReLU()
        # output layer
      self.layer2 = nn.Linear(hidden_size, num_classes)
    
      ### END OF CODE

    def forward(self, x):
     torch.flatten(x, 1)
     x = self.layer1(x)
     x = self.relu(x)
     x = self.layer2(x)
     return x


In [ ]:
net = Net()

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

BATCH_SIZE = 64
EPOCH = 200

torch_dataset = Data.TensorDataset(x, y)

loader = Data.DataLoader(
    dataset=torch_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, num_workers=2,)


# start training
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(loader):
        
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = net(b_x)     # input x and predict based on x

        loss = loss_func(prediction, b_y)

        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
# ax.set_title('Regression Analysis - model 3, Batches', fontsize=35)
ax.set_xlabel('x', fontsize=24)
ax.set_ylabel('sin(x)', fontsize=24)
ax.set_xlim(-11.0, 13.0)
ax.set_ylim(-1.1, 1.2)
ax.scatter(x.data.numpy(), y.data.numpy(), color = "orange", alpha=0.2)
prediction = net(x)     # input x and predict based on x
ax.scatter(x.data.numpy(), prediction.data.numpy(), color='green', alpha=0.5)
plt.show()

## Q4. Corner detection using CNNs

In this section we will implement a Convolution Neural Network(CNN) for detecting corners in images. In particular given an image with a corner in it, the CNN will output the coordinates of the corner in the image.



In [ ]:
import pickle
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
### Only run this cell if you're mounting google drive and using it to acccess 
### the data. If using please change the path accordingly
import os
os.chdir('/content/drive/My Drive/.....')

In [ ]:
data, labels = pickle.load(open('./corner_dataset_100x100.pick', 'rb'))

In [ ]:
tensor_x = torch.Tensor(data) # transform to torch tensor
tensor_y = torch.Tensor(labels)

dataset = TensorDataset(tensor_x,tensor_y) # create your datset
train_split, test_split = int(0.75 * len(dataset)), int(0.25 * len(dataset) + 1) 
print(train_split, test_split, len(dataset))
trainset, testset = torch.utils.data.random_split(dataset, [train_split, test_split])

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)
# dataloader = DataLoader(dataset, batch_size =128, shuffle = True) # create your dataloader

### Q 4.1 Implement the CNN [10 points]
Complete the code below to implement your model. You will have design your own CNN that can detect corners in images. You can find the list of layers available as part of pytorch [here](https://pytorch.org/docs/stable/nn.html). You will use a combination of the following layers:
- Convolution layer - this layer performs convolution on the images. [doc](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d)
- MaxPool Layer - maxpool layers are used to reduce the dimensions of feature maps while maintaining translation invariance. [DOC](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d)
- Flatten - Since the output of the convolution layers are image like feature maps you will need a layer which can convert or flatten the image into a single array. This layer does that. [DOC](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html#torch.nn.Flatten)
- Non Linearities - You need a non-linear activation function after each layer. You have several choices for these such as sigmoid, relu etc. [DOC](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)

In [ ]:
# this is one way to define a network
class CornerNet(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(CornerNet, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1,32, kernel_size=(3,3), stride=1, padding=1)        
        # ... continue the construction of the network
        ### YOUR CODE HERE
        # Hint: You may need torch.nn.Conv2d(), torch.nn.MaxPool2D(), torch.nn.Flatten(), torch.nn.Linear(), torch.nn.Sigmoid(), torch.nn.ReLU()
        self.main = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32 , 64, 5,stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = nn.Sequential(
            nn.Linear(3136, num_classes)
        )
        
        ### END OF CODE

    def forward(self, x):
      x = self.conv1(x)
      ### YOUR CODE HERE
      # make sure to be consistent with your network definition in __init__() function of this class  
      x = self.first(x)
      x = self.main(x)
      x = x.reshape(x.size(0), -1)
      x = self.fc(x)
      ### END OF CODE

      return x
        

net = CornerNet((1, 28, 28), 10)
print(net)

Feel free to change and explore hyperparameters such as learning rate (lr) and number of epochs. You may use different optimizer or criterions as well to improve your accuracy.

In [ ]:
lr = 3e-3
optimizer = torch.optim.Adagrad(net.parameters(), lr = lr)
criterion = torch.nn.MSELoss()
cuda_status = torch.cuda.is_available()

if cuda_status:
  net = net.cuda()
  
net.train()
for epoch in range(80):
  training_loss = 0
  for i,(x,y) in enumerate(trainloader):
    y /= 100
    if cuda_status:
      y = y.cuda()
      x = x.cuda()
    x = torch.unsqueeze(x,1)
    out = net(x)
    loss = criterion(out, y)
    training_loss += loss.item()
    loss.backward()
    optimizer.step()
  training_loss /= i
  print("\tEpoch: {} Loss: {}".format(epoch, training_loss))


We will now evaluate our trained network on the test dataset. Please report the final test loss your network achieves on the data

In [ ]:
net.cpu()
net.eval()
with torch.no_grad():
  test_loss = 0
  for i,(x,y) in enumerate(testloader):
    y /= 100
    x = torch.unsqueeze(x, 1)
    coords = net(x)
    loss = criterion(y, coords).item()
    test_loss += loss
  test_loss /= i
  print("Test loss: ", test_loss)

### Visualize performance

Let's try to visualize the performance of the model. We will plot the image first and then overlay the coordinates of the corner on top of the image to see if the location is correct

In [ ]:
with torch.no_grad():
  for i,(x,y) in enumerate(testloader):
    x = torch.unsqueeze(x, 1)
    coords = net(x)

    for j in range(10):
      fig, ax = plt.subplots()
      ax.imshow(x[j][0], cmap=plt.cm.gray)
      pts = y[j]
      pts = coords[j] * 100
      print(pts)
      print(y[j])
      ax.plot(pts[1], pts[0], color='red', marker='o',
              linestyle='None', markersize=6)
      plt.show()

    break